In [14]:
%load_ext autoreload

%autoreload 2

from __future__ import print_function, division

import cPickle as pickle
from glob import glob
import subprocess
import sys
import string
import os
import random
import numpy as np
from os.path import join
import scipy.linalg

import threebody

import logging
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)
logging.debug("test")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
basic_params = dict(files="tuned2_aligned_pulses_selected",
                     tzrmjd_middle='auto',
                     parfile="0337_de430.par",
                     fit_pos=True,
                     fit_pm=True,
                     fit_px=True,
                     efac=1,
                     t2_astrometry=True,
                     kopeikin=False,
                     ppn_mode='heavysimple',
                     linear_jumps=True,
                     linear_dm=False,
                     fdn_range=(1,5),
                     priors=('dbeta','dgamma'),
                     toa_mode="pipeline",
                     dmx_span=365.2425/2,
                     variable_dm=True,
                     variable_ipm=True)
F = threebody.Fitter(**basic_params)

## Analytical marginalization correction

When we're doing MCMC, we actually simplify the problem by handling the linear parts separately. Specifically, for each set of orbital parameters, we do a linear least-squares fit for all the linear parts and return the log-likelihood of the best-fit value. For this to be proper MCMC, we need to think of this as analytically marginalizing over the possible linear-part values. This requires us to add a correction to the log-probability, which is an integral over all linear-part values. If the fit matrix is $A$, then this correction term is $\log(\det(A^TA))$. Currently we compute this by multplying out $A^TA$ and calling the numpy function `slogdet`. This doesn't operate in long double, the product $A^TA$ has a worse condition number than $A$ itself, and we know the condition number of $A$ can be improved by rescaling its columns. So let's try to do better.

### Just a check: let's re-derive the analytical marginalization.

Our problem is parameterized by a pair of vectors $(x,y)$, and the log-likelihood of the data $d$ given $(x,y)$ is calculated using $L(x,y)=\log(|F(x)y-d|^2/2)$. But we are only really interested in the distribution of $x$, so we want to construct an $L(x)$ that gives rise to the same posterior distribution of $x$ values.  

### Improving the calculation

If we have a matrix $A$, the SVD gives us $A=USV^T$, and the eigenvalues of $A^TA$ are the squares of the diagonal values of $S$. So we can simplify the calculation.


In [16]:
A = np.random.randn(10,4).astype(np.longdouble)

s = scipy.linalg.svdvals(A)

print(np.linalg.slogdet(np.dot(A.T,A).astype(np.float)))
print(np.sum(np.log(s))*2)

(1.0, 8.2541645702244448)
8.25416457022


In [18]:
A, names = F.compute_linear_matrix()
A /= F.phase_uncerts[:,None]

In [19]:
np.linalg.slogdet(np.dot(A.T,A).astype(float))

(1.0, 537.75505864568834)

First step is to rescale the columns of $A$ so that they all have the same $L^2$ norm. (It's not clear that this is necessarily the best rescaling but it does seem to help.)

In [22]:
scales = np.sum(A**2,axis=0)
As = A/scales[None,:]
(np.linalg.slogdet(np.dot(As.T,As).astype(float))[1]
 +2*np.sum(np.log(scales)))

537.75505529991886988

In [23]:
s = scipy.linalg.svdvals(As)
2*np.sum(np.log(scales)+np.log(s))

537.75103079184101434

## Kepler improvements

Currently the Keplerian model that translates from orbital elements to positions and masses uses `scipy.optimize.newton` in a few places. Even apart for whatever inaccuracies it has, this only uses double precision. Can we do better easily?

In [24]:
from scipy.optimize import newton

In [40]:
def mean_from_eccentric(e, ecc):
    return ecc-e*np.sin(ecc)
def eccentric_from_mean(e, mean_anomaly):
    """Compute the eccentric anomaly from the mean anomaly                                                             
                                                                                                                       
    Inputs:                                                                                                            
        e - the eccentricity                                                                                           
        mean_anomaly - the mean anomaly                                                                                
                                                                                                                       
    Outputs:                                                                                                           
        eccentric_anomaly - the true anomaly                                                                           
        derivatives - pair of derivatives with respect to the two inputs                                               
    """
    eccentric_anomaly = newton(
            lambda E: E-e*np.sin(E)-mean_anomaly,
            mean_anomaly,
            lambda E: 1-e*np.cos(E),
            tol=1e-20)
    eccentric_anomaly_de = np.sin(eccentric_anomaly)/(1-e*np.cos(eccentric_anomaly))
    eccentric_anomaly_prime = (1-e*np.cos(eccentric_anomaly))**(-1)
    return eccentric_anomaly, [eccentric_anomaly_de, eccentric_anomaly_prime]



In [44]:
m = np.arctan(np.longdouble(1))*1.3
e = np.longdouble(5e-1)
ecc = eccentric_from_mean(e, m)[0]
r = mean_from_eccentric(e,ecc)
r, (r-m)/m, ecc.dtype, r.dtype

(1.0210176124166828374, 0.0, dtype('float128'), dtype('float128'))